In [5]:
# ==========================================
# 🚀 COLAB LAUNCHER for secure_app_final_ms4.py
# ==========================================

!pip install -q streamlit pyngrok bcrypt pyjwt pandas numpy nltk transformers evaluate datasets rouge-score


from pyngrok import ngrok
import textwrap, psutil, time, os

# ==========================================
# 🔐 YOUR NGROK AUTH TOKEN HERE
# ==========================================
NGROK_AUTH_TOKEN = "YOUR AUTH TOKEN HERE"  # ⬅️ Replace with your token
if NGROK_AUTH_TOKEN.strip():
    ngrok.set_auth_token(NGROK_AUTH_TOKEN)
    print("✅ ngrok token set successfully")
else:
    print("⚠️ No ngrok token provided — public URL won't be generated.")

# ==========================================
# ✍️ WRITE FULL APP CODE TO FILE
# ==========================================
app_code = """
# secure_app_final_ms4.py — FINAL FULL VERSION (Colab-ready)
import streamlit as st
import sqlite3, bcrypt, jwt, pandas as pd, random, time, smtplib
from datetime import datetime, timedelta
from email.mime.text import MIMEText
from email.mime.multipart import MIMEMultipart
import numpy as np
import itertools
from nltk.corpus import wordnet

# ========================
# CONFIGURATION
# ========================
SMTP_EMAIL = "likhithalakamraju@gmail.com"  # <-- your Gmail address
SMTP_PASSWORD = "umcu irpc amql uzks"  # <-- your Gmail App Password
SECRET_KEY = "your-super-secret-key"
DB_NAME = "secure_users.db"

st.set_page_config(page_title="LLM AI - Secure Auth (MS4)", layout="wide", initial_sidebar_state="collapsed")

custom_css = '''
<style>
.main .block-container { max-width: 550px; padding-top: 2rem; }
h1 { background-color: #0d6efd; color: white; padding: 1rem; border-radius: 10px 10px 0 0; text-align: left; font-size: 24px; margin-bottom: 0 !important; }
div[data-testid="stTabs"], div.forgot-password-container { border: 1px solid #E0E0E0; border-radius: 0 0 10px 10px; padding: 1.5rem; box-shadow: 0 4px 8px rgba(0,0,0,0.05); background-color: white; }
.stButton>button { width: 100%; background-color: #0d6efd; color: white; border-radius: 6px; }
.stButton>button:hover { background-color: #0b5ed7; }
</style>
'''
st.markdown(custom_css, unsafe_allow_html=True)

# Stable session id for unique keys
if 'sid' not in st.session_state:
    st.session_state.sid = f"{int(time.time()*1000)}_{random.randint(1000,9999)}"
SID = st.session_state.sid

# ======================
# DATABASE
# ======================
def init_db():
    conn = sqlite3.connect(DB_NAME)
    c = conn.cursor()
    c.execute('CREATE TABLE IF NOT EXISTS users (email TEXT PRIMARY KEY, password_hash BLOB, role TEXT, otp TEXT, otp_expiry TEXT, created_at TEXT)')
    c.execute('CREATE TABLE IF NOT EXISTS usage (id INTEGER PRIMARY KEY AUTOINCREMENT, username TEXT, event TEXT, detail TEXT, time TEXT)')
    c.execute('CREATE TABLE IF NOT EXISTS feedback (id INTEGER PRIMARY KEY AUTOINCREMENT, username TEXT, comment TEXT, up INTEGER, down INTEGER, rating INTEGER, time TEXT)')
    c.execute("SELECT * FROM users WHERE email='admin@ai'")
    if not c.fetchone():
        pw = bcrypt.hashpw("Infosys".encode(), bcrypt.gensalt())
        c.execute("INSERT INTO users (email, password_hash, role, created_at) VALUES (?, ?, ?, ?)", ("admin@ai", pw, "Admin", str(datetime.now().date())))
    conn.commit()
    conn.close()

def hash_pw(p): return bcrypt.hashpw(p.encode(), bcrypt.gensalt())
def verify_pw(p, h):
    if isinstance(h, memoryview): h = bytes(h)
    return bcrypt.checkpw(p.encode(), h)

def register_user(email, password, role="General User"):
    conn=sqlite3.connect(DB_NAME); c=conn.cursor()
    c.execute("SELECT * FROM users WHERE email=?", (email,))
    if c.fetchone(): conn.close(); return "Email already exists."
    c.execute("INSERT INTO users (email,password_hash,role,created_at) VALUES (?,?,?,?)", (email, hash_pw(password), role, str(datetime.now().date())))
    conn.commit(); conn.close(); return "User registered successfully!"

def delete_user(email):
    conn = sqlite3.connect(DB_NAME)
    c = conn.cursor()
    c.execute("DELETE FROM users WHERE email=?", (email,))
    conn.commit()
    conn.close()


# ======================
# EMAIL & OTP
# ======================
def send_otp_email(to_email, otp):
    try:
        msg = MIMEMultipart("alternative")
        msg["Subject"] = "Your OTP Code"
        msg["From"] = SMTP_EMAIL
        msg["To"] = to_email
        msg.attach(MIMEText(f"Your OTP is {otp}. It expires in 10 minutes.", "plain"))
        with smtplib.SMTP("smtp.gmail.com", 587) as s:
            s.starttls(); s.login(SMTP_EMAIL, SMTP_PASSWORD); s.sendmail(SMTP_EMAIL, to_email, msg.as_string())
        return True
    except Exception as e:
        st.warning(f"⚠️ Email not sent: {e}")
        return False

def generate_otp(email):
    otp = str(random.randint(100000,999999))
    expiry = datetime.now() + timedelta(minutes=10)
    conn=sqlite3.connect(DB_NAME); c=conn.cursor()
    c.execute("UPDATE users SET otp=?, otp_expiry=? WHERE email=?", (otp, expiry.strftime("%Y-%m-%d %H:%M:%S"), email))
    conn.commit(); conn.close()
    send_otp_email(email, otp)
    return otp

def verify_otp(email, otp, new_password):
    conn=sqlite3.connect(DB_NAME); c=conn.cursor()
    c.execute("SELECT otp,otp_expiry FROM users WHERE email=?", (email,))
    r=c.fetchone()
    if not r: return "User not found."
    stored, exp=r
    if stored==otp and datetime.now()<datetime.strptime(exp,"%Y-%m-%d %H:%M:%S"):
        c.execute("UPDATE users SET password_hash=?, otp=NULL, otp_expiry=NULL WHERE email=?", (hash_pw(new_password), email))
        conn.commit(); conn.close(); return "✅ Password reset successfully!"
    conn.close(); return "❌ Invalid or expired OTP."

# ======================
# LOGS / FEEDBACK
# ======================
def log_usage(u,e): conn=sqlite3.connect(DB_NAME); conn.execute("INSERT INTO usage (username,event,time) VALUES (?,?,?)",(u,e,str(datetime.now().date()))); conn.commit(); conn.close()
def add_feedback(u,c,up,down,r): conn=sqlite3.connect(DB_NAME); conn.execute("INSERT INTO feedback (username,comment,up,down,rating,time) VALUES (?,?,?,?,?,?)",(u,c,int(up),int(down),int(r),str(datetime.now().date()))); conn.commit(); conn.close()
def get_users_df(): conn=sqlite3.connect(DB_NAME); df=pd.read_sql_query("SELECT email,role,created_at FROM users",conn); conn.close(); return df

# ======================
# NLP UTILITIES
# ======================
try:
    from transformers import pipeline
    summarizer = pipeline("summarization", model="t5-small")
except Exception: summarizer=None

def summarize_text(t):
    if summarizer:
        try: return summarizer(t, max_length=120, min_length=30, do_sample=False)[0]["summary_text"]
        except: pass
    s=t.split('.'); return '. '.join(s[:3])

def paraphrase_text(t):
    w=t.split(); return ' '.join(w[len(w)//2:]+w[:len(w)//2]) if len(w)>6 else t

def augment_text(t,m='synonym',n=1):
    if m=='swap':
        w=t.split()
        for _ in range(n):
            i,j=random.randint(0,len(w)-1),random.randint(0,len(w)-1)
            w[i],w[j]=w[j],w[i]
        return ' '.join(w)
    elif m=='delete':
        return ' '.join([x for x in t.split() if random.random()>0.1])
    else:
        return t + " (augmented)"

def simple_grid_search(model_fn,param_grid):
    keys=list(param_grid.keys()); best=None; res=[]
    for vals in itertools.product(*[param_grid[k] for k in keys]):
        p=dict(zip(keys,vals)); s=model_fn(p)
        res.append((p,s));
        if best is None or s>best[1]: best=(p,s)
    return best,res

def dummy_model_fn(p): return float(p.get('beta',1))

# ======================
# DASHBOARDS
# ======================
def user_dashboard(u):
    # Sidebar navigation
    st.sidebar.title(f"👤 {u}")
    section = st.sidebar.radio(
        "Select a section:",
        ["Data Augmentation & Multilevel Tuning", "Summarization & Paraphrasing", "About"],
        key=f"user_menu_{SID}"
    )
    st.sidebar.markdown("---")
    if st.sidebar.button("Logout", key=f"logout_user_{SID}"):
        st.session_state.token = None
        st.rerun()

    # --- Section 1: Data Augmentation (keep your existing code here) ---
    if section == "Data Augmentation & Multilevel Tuning":
        # Disable wandb and SET before any imports!
        import os
        os.environ["WANDB_DISABLED"] = "true"
        os.environ["WANDB_MODE"] = "disabled"
        import time
        import torch
        import evaluate
        import plotly.graph_objects as go
        from datasets import load_dataset, DatasetDict
        from transformers import (
            AutoTokenizer, AutoModelForSeq2SeqLM,
            Trainer, TrainingArguments, DataCollatorForSeq2Seq
        )

        st.markdown("## 🧠 Dataset Augmentation & Multilevel Tuning")

        left, right = st.columns([2,1], gap="large")

        with left:
            st.subheader("📘 Dataset Management & Model Selection")
            ds_choice = st.radio("Choose dataset sample:", ["WikiAuto (small sample)", "CNN/DailyMail (small sample)"])
            selected_ds = "wiki_auto" if "WikiAuto" in ds_choice else "cnn_dailymail"
            st.write("*Selected Dataset:*", ds_choice)
            st.markdown("---")
            st.subheader("⚙ Training Configuration (demo run)")
            # Set very small values for fast demo
            n_train = st.number_input("Train sample size (per split)", min_value=8, max_value=32, value=8, step=8)
            epochs = st.number_input("Epochs", min_value=1, max_value=1, value=1)
            batch_size = st.selectbox("Batch size", [4], index=0)
            lr = st.text_input("Learning rate", value="5e-5")
            start_train = st.button("🚀 Start Training")
            progress_bar = st.progress(0)
            status_box = st.empty()
            completed_box = st.empty()

        with right:
            st.subheader("📈 Live Model Evaluation Panel")
            device = "cuda" if torch.cuda.is_available() else "cpu"
            st.write("Running on:", device)
            model_loaded_box = st.empty()
            st.markdown("---")
            st.subheader("✍ Enter Text to Summarize")
            user_text = st.text_area("Input Text", height=200)
            summarize_button = st.button("🔍 Generate Simplified Text")
            simplified_out = st.empty()
            st.markdown("---")
            st.subheader("📏 Evaluation Metrics")
            col1, col2, col3, col4 = st.columns(4)
            m_bleu = col1.empty()
            m_rouge = col2.empty()
            m_loss = col3.empty()
            m_time = col4.empty()
            st.markdown("---")
            st.subheader("📊 Evaluation Radar")
            radar_slot = st.empty()

        if "tokenizer" not in st.session_state:
            st.session_state["tokenizer"] = None
        if "model" not in st.session_state:
            st.session_state["model"] = None
        if "trained" not in st.session_state:
            st.session_state["trained"] = False

        if start_train:
            try:
                status_box.info("Preparing dataset and model...")
                progress_bar.progress(10)
                # Clear metric slots before training
                m_bleu.empty(); m_rouge.empty(); m_loss.empty(); m_time.empty(); radar_slot.empty()
                if selected_ds == "wiki_auto":
                    ds = load_dataset("GEM/wiki_auto_asset_turk")
                    train_cols = ds["train"].column_names
                    if "document" in train_cols and "summary" in train_cols:
                        src_col, tgt_col = "document", "summary"
                    elif "source" in train_cols and "target" in train_cols:
                        src_col, tgt_col = "source", "target"
                    elif "text" in train_cols and "summary" in train_cols:
                        src_col, tgt_col = "text", "summary"
                    else:
                        lengths = {c: sum(len(str(x)) for x in ds["train"][c][:50]) for c in train_cols}
                        sorted_cols = sorted(lengths.items(), key=lambda x: x[1], reverse=True)
                        src_col, tgt_col = sorted_cols[0][0], sorted_cols[1][0]
                else:
                    ds = load_dataset("cnn_dailymail", "3.0.0")
                    src_col, tgt_col = "article", "highlights"
                status_box.info("Dataset loaded! Preparing sample ...")
                progress_bar.progress(20)
                n = int(n_train)
                train_all = ds["train"].shuffle(seed=42)
                small = train_all.select(range(min(len(train_all), max(8, n*2))))
                split_idx = int(len(small)*0.8)
                train_ds = small.select(range(split_idx))
                val_ds = small.select(range(split_idx, len(small)))
                sampled = DatasetDict({"train": train_ds, "validation": val_ds})
                progress_bar.progress(30)
                status_box.info("Loading tokenizer & model...")
                tokenizer = AutoTokenizer.from_pretrained("t5-small")
                model = AutoModelForSeq2SeqLM.from_pretrained("t5-small")
                st.session_state["tokenizer"] = tokenizer
                st.session_state["model"] = model
                progress_bar.progress(40)
                status_box.info("Tokenizing data...")
                def preprocess(ex):
                    inputs = ["summarize: " + t for t in ex[src_col]]
                    model_inputs = tokenizer(inputs, max_length=256, truncation=True, padding="max_length")
                    with tokenizer.as_target_tokenizer():
                        labels = tokenizer(ex[tgt_col], max_length=128, truncation=True, padding="max_length")
                    model_inputs["labels"] = labels["input_ids"]
                    return model_inputs
                tokenized_train = sampled["train"].map(preprocess, batched=True)
                tokenized_val = sampled["validation"].map(preprocess, batched=True)
                progress_bar.progress(55)
                data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)
                args = TrainingArguments(
                    output_dir="trained_model/demo_run",
                    num_train_epochs=1,
                    per_device_train_batch_size=4,
                    learning_rate=5e-5,
                    logging_steps=10,
                    save_strategy="no",
                    disable_tqdm=True
                )
                trainer = Trainer(
                    model=model, args=args,
                    train_dataset=tokenized_train,
                    eval_dataset=tokenized_val,
                    tokenizer=tokenizer,
                    data_collator=data_collator,
                )
                status_box.info("Training - demo run (should finish under 1 minute with tiny data)...")
                t0 = time.time()
                trainer.train()
                t1 = time.time()
                model.save_pretrained("trained_model/demo_run")
                tokenizer.save_pretrained("trained_model/demo_run")
                completed_box.info(f"Training finished in {t1-t0:.1f}s")
                progress_bar.progress(75)
                status_box.info("Running evaluation...")
                metric_rouge = evaluate.load("rouge")
                metric_bleu = evaluate.load("bleu")
                val_texts = sampled["validation"][src_col]
                val_refs = sampled["validation"][tgt_col]
                preds = []
                eval_start = time.time()
                for txt in val_texts:
                    inp = tokenizer("summarize: " + txt, return_tensors="pt", truncation=True)
                    gen = model.generate(**inp, max_length=128, num_beams=4)
                    preds.append(tokenizer.decode(gen[0], skip_special_tokens=True))
                eval_end = time.time()
                status_box.info("Evaluating metrics...")
                # Removed prediction printouts
                rouge_scores = metric_rouge.compute(predictions=preds, references=val_refs)
                bleu_scores = metric_bleu.compute(predictions=preds, references=val_refs)
                eval_metrics = trainer.evaluate()
                eval_loss = eval_metrics.get("eval_loss", None)
                eval_runtime = eval_end - eval_start
                m_bleu.metric("BLEU", f"{bleu_scores.get('bleu',0):.4f}")
                m_rouge.metric("ROUGE-L", f"{rouge_scores.get('rougeL',0):.4f}")
                m_loss.metric("Eval Loss", f"{eval_loss:.3f}" if eval_loss else "N/A")
                m_time.metric("Runtime (s)", f"{eval_runtime:.1f}")
                r_bleu = bleu_scores.get("bleu",0)
                r_rouge = rouge_scores.get("rougeL",0)
                r_inv_loss = 1/(1+eval_loss) if eval_loss else 0
                r_speed = 1/(1+(eval_runtime/50))
                cats = ["BLEU","ROUGE-L","Inv Loss","Speed"]
                vals = [r_bleu, r_rouge, r_inv_loss, r_speed]
                fig = go.Figure()
                fig.add_trace(go.Scatterpolar(
                    r=vals+[vals[0]],
                    theta=cats+[cats[0]],
                    fill='toself',
                    name="Metrics"
                ))
                fig.update_layout(polar=dict(radialaxis=dict(visible=True, range=[0,1])),
                                  showlegend=False, height=320)
                radar_slot.plotly_chart(fig, use_container_width=True)
                progress_bar.progress(100)
                status_box.success("Evaluation complete!")
            except Exception as e:
                status_box.error(f"Error during training/eval: {e}")

        if summarize_button:
            tokenizer = st.session_state["tokenizer"] or AutoTokenizer.from_pretrained("t5-small")
            model = st.session_state["model"] or AutoModelForSeq2SeqLM.from_pretrained("t5-small")

            if not user_text.strip():
                simplified_out.warning("Enter some text first.")
            else:
                with st.spinner("Generating..."):
                    inp = tokenizer("summarize: " + user_text, return_tensors="pt", truncation=True)
                    gen = model.generate(**inp, max_length=120, min_length=20, num_beams=4)
                    summary = tokenizer.decode(gen[0], skip_special_tokens=True)

                # 💥 This line prevents duplication
                simplified_out.empty()

                # show summary only ONCE
                simplified_out.success(summary)

    # --- Section 2: Summarization & Paraphrasing (merged page) ---
    elif section == "Summarization & Paraphrasing":
        st.title("📝 Summarization & Paraphrasing")
        st.write("Summarize or Paraphrase text side by side — and share your feedback below.")

        col1, col2 = st.columns(2)

        # Left side - Summarization
        with col1:
            st.subheader("🧾 Text Summarization")
            s = st.text_area("Enter text to summarize", key=f"s_{SID}")
            if st.button("Summarize", key=f"sum_{SID}"):
                if s.strip():
                    summary = summarize_text(s)
                    st.success("✅ Summary:")
                    st.write(summary)
                    log_usage(u, "summarize")
                else:
                    st.warning("Please enter some text to summarize.")

        # Right side - Paraphrasing
        with col2:
            st.subheader("🔄 Text Paraphrasing")
            p = st.text_area("Enter text to paraphrase", key=f"p_{SID}")
            if st.button("Paraphrase", key=f"para_{SID}"):
                if p.strip():
                    paraphrased = paraphrase_text(p)
                    st.success("✅ Paraphrased Text:")
                    st.write(paraphrased)
                    log_usage(u, "paraphrase")
                else:
                    st.warning("Please enter text to paraphrase.")

        # Feedback form inside a bordered card
        with st.container():
            st.markdown(
                "<div style='border:2px solid #ddd; border-radius:12px; padding:20px; background-color:white; margin-top:15px;'>",
                unsafe_allow_html=True
            )

            c = st.text_area("✍️ Write your feedback here:", key=f"c_{SID}", height=120)

            st.markdown("##### How satisfied are you with the app?")
            col1, col2, col3, col4, col5 = st.columns(5)
            with col1:
                r1 = st.button("😡", key=f"r1_{SID}")
            with col2:
                r2 = st.button("😕", key=f"r2_{SID}")
            with col3:
                r3 = st.button("😐", key=f"r3_{SID}")
            with col4:
                r4 = st.button("😊", key=f"r4_{SID}")
            with col5:
                r5 = st.button("🤩", key=f"r5_{SID}")

            # Map emoji buttons to rating values
            rating = 0
            if r1: rating = 1
            elif r2: rating = 2
            elif r3: rating = 3
            elif r4: rating = 4
            elif r5: rating = 5

            st.markdown("##### Overall Impression")
            colx, coly = st.columns(2)
            up = colx.button("👍 Like", key=f"up_{SID}")
            down = coly.button("👎 Dislike", key=f"down_{SID}")

            st.markdown("<br>", unsafe_allow_html=True)
            if st.button("🚀 Submit Feedback", use_container_width=True, key=f"sub_{SID}"):
                if c.strip():
                    add_feedback(u, c, up, down, rating)
                    st.success("✅ Thank you for your valuable feedback!")
                else:
                    st.warning("Please write a comment before submitting.")

            st.markdown("</div>", unsafe_allow_html=True)

    # --- Section 3: About ---
    elif section == "About":
        st.title("🧩 About This App")
        st.info("This section shows deployment details and feedback options.")
        st.success("✅ Application is live and running")
        public_url = "https://textsummarizer-demo.huggingface.co"
        st.link_button("Open Application", public_url)
        st.markdown("---")
        st.metric("Hugging Face", "v2.1.0", "Platform")
        st.metric("Docker", "99.8%", "Uptime")
        st.markdown("---")


def admin_dashboard():
    if st.sidebar.button("Logout", key=f"logout_admin_{SID}"): st.session_state.token=None; st.rerun()
    st.title("Admin Dashboard")
    t1,t2,t3=st.tabs(["Users","Analytics","History"])
    with t1:
            st.subheader("👥 User Accounts")

            df = get_users_df()

    # Show user table with delete button
            if not df.empty:
                for index, row in df.iterrows():
                    col1, col2, col3, col4 = st.columns([4, 2, 2, 1])
                    col1.write(row['email'])
                    col2.write(row['role'])
                    col3.write(row['created_at'])

            # Prevent deleting admin account
                    if row['email'] != "admin@ai":
                        if col4.button("🗑 Delete", key=f"del_{row['email']}"):
                            delete_user(row['email'])
                            st.success(f"User {row['email']} deleted successfully.")
                            st.rerun()
                    else:
                        col4.write("🔒")

            st.markdown("---")
            st.subheader("➕ Add New User")

            e = st.text_input("Email", key=f"addE_{SID}")
            p = st.text_input("Password", type="password", key=f"addP_{SID}")
            r = st.selectbox("Role", ["General User", "Admin"], key=f"addR_{SID}")

            if st.button("Add", key=f"addB_{SID}"):
                st.write(register_user(e, p, r))

    with t2:
        dfu=pd.read_sql_query("SELECT username,event,time FROM usage",sqlite3.connect(DB_NAME))
        if not dfu.empty:
            counts = dfu['event'].value_counts()
            st.subheader("📊 Usage Distribution")
            st.bar_chart(counts)
            st.subheader("📈 Daily Usage Trend")
            st.line_chart(dfu['time'].value_counts().sort_index())
        else: st.write("No data yet.")
    with t3:
            st.subheader("📜 Full Usage History")
            conn = sqlite3.connect(DB_NAME)
            ue = pd.read_sql_query("SELECT username, event, detail, time FROM usage ORDER BY time DESC", conn)
            conn.close()

            if ue.empty:
              st.info("No usage history yet.")
            else:
              st.dataframe(ue, use_container_width=True)

            st.subheader("⭐ User Feedback History")
            conn = sqlite3.connect(DB_NAME)
            fb = pd.read_sql_query("SELECT username, comment, rating, time FROM feedback ORDER BY time DESC", conn)
            conn.close()

            if not fb.empty:
              st.dataframe(fb, use_container_width=True)
            else:
              st.info("No feedback records yet.")


# ======================
# AUTHENTICATION
# ======================
def gen_token(e,r): return jwt.encode({'sub':e,'role':r,'exp':datetime.utcnow()+timedelta(hours=2)},SECRET_KEY,algorithm='HS256')
def dec_token(t):
    try: return jwt.decode(t,SECRET_KEY,algorithms=['HS256'])
    except: return None

init_db()
if 'token' not in st.session_state: st.session_state.token=None
p=dec_token(st.session_state.token)

if p is None:
    st.header("User Authentication")
    tab1,tab2=st.tabs(["Login","Register"])
    with tab1:
        e=st.text_input("Email",key=f"LE_{SID}").lower(); pw=st.text_input("Password",type="password",key=f"LP_{SID}")
        if st.button("Login",key=f"LB_{SID}"):
            conn=sqlite3.connect(DB_NAME); c=conn.cursor(); c.execute("SELECT password_hash,role FROM users WHERE email=?",(e,)); r=c.fetchone(); conn.close()
            if r and verify_pw(pw,r[0]): st.session_state.token=gen_token(e,r[1]); log_usage(e,"login"); st.rerun()
            else: st.error("Invalid login.")
        if st.button("Forgot Password",key=f"FP_{SID}"): st.session_state.auth_view="reset"; st.rerun()
    with tab2:
        ne=st.text_input("Email",key=f"NE_{SID}").lower()
        npw=st.text_input("Password",type="password",key=f"NP_{SID}")
        cnpw=st.text_input("Confirm Password",type="password",key=f"CP_{SID}")

        if st.button("Register",key=f"RB_{SID}"):
            if npw==cnpw:
                  st.success(register_user(ne, npw, "General User"))
            else:
                  st.error("Passwords do not match.")

else:
    r=p['role']; u=p['sub']; st.sidebar.success(f"Logged in as {u} ({r})")
    if r.lower().startswith("admin"): admin_dashboard()
    else: user_dashboard(u)
"""

with open("secure_app_final_ms4.py", "w") as f:
    f.write(app_code)

print("✅ App code written to secure_app_final_ms4.py")

# ==========================================
# 🚀 LAUNCH STREAMLIT APP VIA NGROK
# ==========================================
# Kill any previous Streamlit processes
for p in psutil.process_iter():
    try:
        if "streamlit" in " ".join(p.cmdline()):
            p.kill()
    except Exception:
        pass

# Run Streamlit
get_ipython().system_raw(
    "nohup streamlit run secure_app_final_ms4.py --server.port 8501 --server.enableCORS false --server.headless true > logs.txt 2>&1 &"
)
time.sleep(3)

if NGROK_AUTH_TOKEN.strip():
    public_url = ngrok.connect(8501)
    print("🚀 Streamlit App Running at:", public_url.public_url)
    print("➡️ Default admin login: admin@ai / Infosys")
else:
    print("⚠️ Streamlit started locally. Use Colab preview or set NGROK_AUTH_TOKEN for public link.")

✅ ngrok token set successfully
✅ App code written to secure_app_final_ms4.py


🚀 Streamlit App Running at: https://hydrotactic-carletta-fremdly.ngrok-free.dev
➡️ Default admin login: admin@ai / Infosys
